In [1]:
import requests
import pyspark
from pyspark.sql.functions import split, col,substring,regexp_replace
from pyspark.sql import SparkSession
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext 
import pyspark.sql.functions as func
from pyspark.sql.functions import isnull,when,count 
from pyspark.sql.functions import lower, col, lit
from pyspark.sql import functions as f
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
import findspark 
findspark.init() 

To download Dataframe 

In [ ]:
#Downloaded dataframe from URL
#url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
#req_url = requests.get(url)
#open('Online_shopping_analysis.xlsx', 'wb').write(req_url.content)

In [ ]:
#read excel file
#df_excel = pd.read_excel('Online_shopping_analysis.xlsx')

Conversion of Online_shopping_analysis Dataframe from Excel into CSV

In [ ]:
#converting excel file into csv
#df_excel.to_csv('Online_shopping_analysis.csv', index = None, header=True)

In [5]:
input_uri="mongodb://127.0.0.1/test.project_final"
output_uri="mongodb://127.0.0.1/test.project_final"
mongodb_spark_jars_path="/home/hduser/mongojars/"

In [6]:
spark=SparkSession.builder.appName("Project")\
    .config("spark.mongodb.input.uri",input_uri)\
    .config("spark.mongodb.output.uri",output_uri)\
    .config("spark.jars",""+mongodb_spark_jars_path+"mongo-spark-connector_2.12-3.0.1.jar,"+mongodb_spark_jars_path+"bson-4.0.5.jar,"+mongodb_spark_jars_path+"mongodb-driver-core-4.0.5.jar,"+mongodb_spark_jars_path+"mongodb-driver-sync-4.0.5.jar")\
    .getOrCreate()

/home/hduser/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [15]:
Online_shopping_analysis_df = spark.read.option("delimiter", ",").option("Inferschema",'true').option("header","true").csv("Online_shopping_analysis.csv")

In [16]:
Online_shopping_analysis_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [17]:
#Check count of rows and columns
print((Online_shopping_analysis_df.count(), len(Online_shopping_analysis_df.columns)))

(541909, 8)


In [18]:
#To check null values present in dataframe
Online_shopping_analysis_df.select([count(when(isnull(c),c)).alias(c) for c in Online_shopping_analysis_df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [19]:
#Removing null values
remove_null = Online_shopping_analysis_df.dropna()

In [20]:
#all null, blank values are removed
remove_null.select([count(when(isnull(c),c)).alias(c) for c in remove_null.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|         0|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [21]:
#Check count of rows and columns
print((remove_null.count(), len(remove_null.columns)))

(406829, 8)


In [22]:
#Extracting non-zero and positive values from UnitPrice column
extract_df = remove_null.filter((remove_null.UnitPrice != 0 ) | (remove_null.UnitPrice > 0))

In [23]:
#Removing Duplicate rows
distinct_rows = extract_df.distinct()

In [24]:
# Extracting positive quantity
positive_qty = distinct_rows.filter(distinct_rows.Quantity > 0)

In [25]:
#Extraction of Hours from date column
hours_df=(positive_qty.withColumn("Hours",f.hour("InvoiceDate")))
hours_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|Hours|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----+
|   536408|    84879|ASSORTED COLOUR B...|       8|2010-12-01 11:41:00|     1.69|   14307.0|United Kingdom|   11|
|   536409|    22074|6 RIBBONS SHIMMER...|       1|2010-12-01 11:45:00|     1.65|   17908.0|United Kingdom|   11|
|   536409|   90199C|5 STRAND GLASS NE...|       2|2010-12-01 11:45:00|     6.35|   17908.0|United Kingdom|   11|
|   536488|    22150|3 STRIPEY MICE FE...|       1|2010-12-01 12:31:00|     1.95|   17897.0|United Kingdom|   12|
|   536522|    22566|FELTCRAFT HAIRBAN...|       1|2010-12-01 12:49:00|     0.85|   15012.0|United Kingdom|   12|
|   536523|    21259|VICTORIAN SEWING ...|       2|2010-12-01 12:50:00|     5.95|   1286

In [26]:
#Extraction of date in the format of YYYY-mm-dd from date column
date_df = (hours_df.withColumn("InvoiceDate",f.to_date("InvoiceDate")))
date_df.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|Hours|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----+
|   536408|    84879|ASSORTED COLOUR B...|       8| 2010-12-01|     1.69|   14307.0|United Kingdom|   11|
|   536409|    22074|6 RIBBONS SHIMMER...|       1| 2010-12-01|     1.65|   17908.0|United Kingdom|   11|
|   536409|   90199C|5 STRAND GLASS NE...|       2| 2010-12-01|     6.35|   17908.0|United Kingdom|   11|
|   536488|    22150|3 STRIPEY MICE FE...|       1| 2010-12-01|     1.95|   17897.0|United Kingdom|   12|
|   536522|    22566|FELTCRAFT HAIRBAN...|       1| 2010-12-01|     0.85|   15012.0|United Kingdom|   12|
|   536523|    21259|VICTORIAN SEWING ...|       2| 2010-12-01|     5.95|   12868.0|United Kingdom|   12|
|   536528|    22635|CHILDS BREAKFAST ...|    

In [27]:
#Extracting features from date
date_df=(date_df.withColumn("year",f.year("InvoiceDate")))
date_df=(date_df.withColumn("month",f.month("InvoiceDate")))
date_df=(date_df.withColumn("dayofmonth",f.dayofmonth("InvoiceDate")))
date_df=(date_df.withColumn("dayofweek",f.dayofweek("InvoiceDate")))
date_df=(date_df.withColumn("dayofyear",f.dayofyear("InvoiceDate")))
date_df=(date_df.withColumn("weekofyear",f.weekofyear("InvoiceDate")))
date_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- Hours: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- dayofyear: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)



In [28]:
date_df.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----+----+-----+----------+---------+---------+----------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|Hours|year|month|dayofmonth|dayofweek|dayofyear|weekofyear|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+-----+----+-----+----------+---------+---------+----------+
|   536408|    84879|ASSORTED COLOUR B...|       8| 2010-12-01|     1.69|   14307.0|United Kingdom|   11|2010|   12|         1|        4|      335|        48|
|   536409|    22074|6 RIBBONS SHIMMER...|       1| 2010-12-01|     1.65|   17908.0|United Kingdom|   11|2010|   12|         1|        4|      335|        48|
|   536409|   90199C|5 STRAND GLASS NE...|       2| 2010-12-01|     6.35|   17908.0|United Kingdom|   11|2010|   12|         1|        4|      335|        48|
|   536488|    22150|3 STRIPEY MICE FE...|    

In [29]:
#remove "Unspecified" country from country column
remove_Unspecified = date_df.filter((date_df.Country != 'Unspecified' ))

In [30]:
#Replacing country column values Channel Islands & European Community with one of the country names
replace_country = remove_Unspecified.na.replace(['Channel Islands','European Community'], ['Jersey','Germany'], 'Country')

In [31]:
#created totalprice column by doing product of Quantity & UnitPrice
totalprice = replace_country.select("*",(col("Quantity") * col("UnitPrice")).alias("Total_Price"))

In [32]:
#Checking total columns 
totalprice.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'Hours',
 'year',
 'month',
 'dayofmonth',
 'dayofweek',
 'dayofyear',
 'weekofyear',
 'Total_Price']

In [33]:
#checking count of rows after processing
totalprice.count()

392451

# Extraction of public_holidays Dataframe into CSV

In [2]:
#Downloaded datafreame from URL
url = 'https://data.worldpop.org/GIS/Holiday_Data/public_holidays/public_holidays_2010_2019.zip'
req = requests.get(url)
open('public_holidays_2010_2019.zip', 'wb').write(req.content)

245112

In [3]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "public_holidays_2010_2019.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    zip.extractall()


File Name                                             Modified             Size
public_holidays_2010_2019.csv                  2019-11-25 22:48:54      2550319


In [7]:
public_holidays_df = spark.read.option("delimiter", ",").option("Inferschema",'true').option("header","true").csv("public_holidays_2010_2019.csv")

In [8]:
public_holidays_df.printSchema()

root
 |-- ADM_name: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type: string (nullable = true)



In [9]:
public_holidays_df.show()

+--------+----+----------+--------------------+--------------+
|ADM_name|ISO3|      Date|                Name|          Type|
+--------+----+----------+--------------------+--------------+
|   Aruba| ABW|01/01/2010|      New Year's Day|Public holiday|
|   Aruba| ABW|25/01/2010|Betico Croes' Bir...|Public holiday|
|   Aruba| ABW|15/02/2010|     Carnival Monday|Public holiday|
|   Aruba| ABW|18/03/2010|National Anthem a...|Public holiday|
|   Aruba| ABW|02/04/2010|         Good Friday|Public holiday|
|   Aruba| ABW|05/04/2010|       Easter Monday|Public holiday|
|   Aruba| ABW|27/04/2010|          King's Day|Public holiday|
|   Aruba| ABW|01/05/2010|           Labor Day|Public holiday|
|   Aruba| ABW|13/05/2010|       Ascension Day|Public holiday|
|   Aruba| ABW|25/12/2010|       Christmas Day|Public holiday|
|   Aruba| ABW|26/12/2010|          Boxing Day|Public holiday|
|   Aruba| ABW|01/01/2011|      New Year's Day|Public holiday|
|   Aruba| ABW|25/01/2011|Betico Croes' Bir...|Public h

In [10]:
#To check null values present in dataframe
public_holidays_df.select([count(when(isnull(c),c)).alias(c) for c in public_holidays_df.columns]).show()

+--------+----+----+----+----+
|ADM_name|ISO3|Date|Name|Type|
+--------+----+----+----+----+
|       0|   0|   0|   0|   0|
+--------+----+----+----+----+



In [11]:
#Drop columns ISO3 & Name
columns_to_drop = ['ISO3', 'Name']
column_drop= public_holidays_df.drop(*columns_to_drop)

In [12]:
#creating function for date conversion
from datetime import datetime
import pyspark.sql.types as T
import pyspark.sql.functions as F


def udf_timestamp(Date):
    _date = datetime.strptime(Date, '%d/%m/%Y')
    return _date.strftime('%Y-%m-%d')

udf_date = F.udf(udf_timestamp, T.StringType())

In [14]:
#change date format from dd/mm/YYYY to YYYY-mm-dd
date_conversion = column_drop.withColumn('Date', udf_date('Date'))
date_conversion.show()

+--------+----------+--------------+
|ADM_name|      Date|          Type|
+--------+----------+--------------+
|   Aruba|2010-01-01|Public holiday|
|   Aruba|2010-01-25|Public holiday|
|   Aruba|2010-02-15|Public holiday|
|   Aruba|2010-03-18|Public holiday|
|   Aruba|2010-04-02|Public holiday|
|   Aruba|2010-04-05|Public holiday|
|   Aruba|2010-04-27|Public holiday|
|   Aruba|2010-05-01|Public holiday|
|   Aruba|2010-05-13|Public holiday|
|   Aruba|2010-12-25|Public holiday|
|   Aruba|2010-12-26|Public holiday|
|   Aruba|2011-01-01|Public holiday|
|   Aruba|2011-01-25|Public holiday|
|   Aruba|2011-03-07|Public holiday|
|   Aruba|2011-03-18|Public holiday|
|   Aruba|2011-04-22|Public holiday|
|   Aruba|2011-04-25|Public holiday|
|   Aruba|2011-04-27|Public holiday|
|   Aruba|2011-05-01|Public holiday|
|   Aruba|2011-06-02|Public holiday|
+--------+----------+--------------+
only showing top 20 rows



In [34]:
#Extracting data from 2010-12-01 to 2011-12-09
extract_date = date_conversion.filter(F.col("Date").between('2010-12-01','2011-12-09'))

In [35]:
#replace country names
ADM_name_replace = extract_date.na.replace(['Ukraine', 'Uk', 'Ireland','Czech','Hong-kong','Russia','Saudi-arabia','United-arab-emirates'], ['Ukraine', 'United Kingdom','EIRE','Czech Republic','Hong Kong','RSA','Saudi Arabia','United Arab Emirates'], 'ADM_name')

In [36]:
#replacing types of holidays into "Holiday"
replace_Type = ADM_name_replace.withColumn("Type", F.when((F.col("Type").isin("Public holiday","Half-day holiday","Local holiday","Local observance","Observance","Special holiday")),"Holiday"))

In [39]:
#Merging two datasets
New_df = [totalprice.Country == replace_Type.ADM_name, totalprice.InvoiceDate == replace_Type.Date]
merged_df = totalprice.join(replace_Type, New_df, 'leftouter')

In [40]:
merged_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- Hours: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- dayofyear: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- Total_Price: double (nullable = true)
 |-- ADM_name: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Type: string (nullable = true)



In [ ]:
#Replacing null values with weekday
weekday_df = merged_df.na.fill(value="WeekDay",subset=["Type"])

In [ ]:
#creating dataset for machine learning & powerBI
final_df = weekday_df.drop('ADM_name','Date')

In [ ]:
#Dumping data in the mongoDB
final_df.write.format("com.mongodb.spark.sql.DefaultSource").option("collection", "project_final").mode("Overwrite").save()